# Block Encoding of a hermitian matrix A

Let's see how the BE of an matrix A is computed through the different methods explored in BE_general.ipynb

The Hermitian matrix that is going to be used as example for numerical results is the following:

$$ 
A =
\begin{pmatrix} 
3 & 0 & 6  & 2 \\ 
0 & 1 & 0  & 1 \\ 
6 & 0 & -2 & 0 \\ 
2 & 1 & 0  & 4 
\end{pmatrix} 
$$

In the end we will get:

$$
U_A = 
\begin{pmatrix}
A / \alpha & * \\
* & *
\end{pmatrix}
$$



---
(something wrong in 2.1)
(2.2 omitted)

In [24]:
import numpy as np
import pandas as pd

import pennylane as qml
from IPython.display import Image, display
import matplotlib.pyplot as plt
from pennylane.templates.state_preparations.mottonen import compute_theta, gray_code


from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, Statevector
from scipy.linalg import svd, sqrtm

A = np.array(
    [[3,  0,  6,  2],
     [0,  1,  0,  1j],
     [6,  0, -2,  0],
     [2,  -1j,  0,  4]], dtype=np.complex128)   
     

## 1. LCU (Pennylane)

In [25]:
LCU = qml.pauli_decompose(A)   
LCU_coeffs, LCU_ops = LCU.terms()

# Calculate how many target qubits n are going to be needed for this example
n = int(np.log2(A.shape[0]))            # A is 2^n x 2^n

# Had to add this absolute value because some coeffs were negative. 
weights = np.abs(LCU_coeffs)
alphas  = np.sqrt(weights) / np.linalg.norm(np.sqrt(weights))    # PREP amplitudes from absolute coefficients
# Then rescue the sign in the unitaries
def signed_op(op, c):  # Fold signs into the unitaries: U_i' = sign(c_i) * U_i 
    return op if c >= 0 else qml.s_prod(-1.0, op)  # -U is also unitary

# Calculate how many ancilla qubits m are going to be needed for this example
k = len(LCU_ops)
m = int(np.ceil(np.log2(k))) # 2^m >= number of terms in LCU
assert 2**m >= k, "Not enough ancilla qubits for the LCU terms"

# Define wires (targets after ancillas) and remap 
ancilla_wires = list(range(m))               # [0..m-1]
target_wires  = list(range(m, m+n))          # [m .. m+n-1]
mapping = {i: target_wires[i] for i in range(n)}
unitaries = [qml.map_wires(signed_op(op,c), mapping) for c, op in zip(LCU_coeffs,LCU_ops)]

# Pad to a power of two, both unitaries and amplitudes (with zeros and identities)
if k < 2**m:
    pad = 2**m - k
    unitaries += [qml.Identity(wires=target_wires)] * pad
    alphas = np.concatenate([alphas, np.zeros(pad)])


dev1 = qml.device("default.qubit", wires = m+n)
@qml.qnode(dev1)                                
def lcu_circuit():                              
    # PREP
    qml.StatePrep(alphas, ancilla_wires)             

    # SEL
    qml.Select(unitaries, control=ancilla_wires)         

    # PREP_dagger
    qml.adjoint(qml.StatePrep(alphas, ancilla_wires))
    return qml.state()                           

U1 = qml.matrix(lcu_circuit)()
print("Block-encoded A.1:\n",np.round(U1,4))



# The block-encoding scale is alpha = sum |c_i|
alpha1 = np.sum(weights)
print("\nalpha (sum |c_i|) =", alpha1)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = 2**n
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block1 = U1[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block1, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha1 * block1, 6))

Block-encoded A.1:
 [[ 0.2143+0.j      0.    +0.j      0.4286-0.j     ...  0.    +0.j
  -0.0704+0.0715j  0.081 +0.j    ]
 [ 0.    +0.j      0.0714+0.j     -0.    +0.j     ...  0.    +0.j
  -0.001 +0.j     -0.0096-0.0106j]
 [ 0.4286+0.j     -0.    +0.j     -0.1429+0.j     ... -0.001 +0.j
   0.0905+0.j      0.    +0.j    ]
 ...
 [ 0.    +0.j      0.    +0.j     -0.001 +0.j     ...  0.8692+0.j
  -0.0012+0.j      0.0031+0.0731j]
 [-0.0704-0.0715j -0.001 +0.j      0.0905+0.j     ... -0.0012+0.j
   0.8119+0.j      0.    +0.j    ]
 [ 0.081 +0.j     -0.0096+0.0106j  0.    +0.j     ...  0.0031-0.0731j
   0.    +0.j      0.8692+0.j    ]]

alpha (sum |c_i|) = 14.0

Approx A/alpha block (real part):
[[ 0.214286+0.j        0.      +0.j        0.428571-0.j
   0.142857+0.j      ]
 [ 0.      +0.j        0.071429+0.j       -0.      +0.j
   0.      +0.071429j]
 [ 0.428571+0.j       -0.      +0.j       -0.142857+0.j
   0.      +0.j      ]
 [ 0.142857+0.j        0.      -0.071429j  0.      +0.j
   0.28571

## 2. Matrix Access Oracle (Pennylane)
### 2.1 Structured matrix

In [26]:
# SOMETHING WRONG HERE, REVISE!

alphas = np.arccos(A).flatten() 
thetas = compute_theta(alphas)

ancilla_wires = ["ancilla"]

s = int(np.log2(A.shape[0]))                   # log2 of the number of rows/columns of A
wires_i = [f"i{index}" for index in range(s)]   # ['i0', 'i1'] for s=2
wires_j = [f"j{index}" for index in range(s)]   # ['j0', 'j1'] for s=2

# Then obtain the control wires for the C-NOT gates and a wire map that we later use to translate the control wires into the wire registers we prepared.
code = gray_code(int(2 * np.log2(len(A))))      # ['0000', '0001', '0011', '0010', '0110', '0111', '0101', '0100', '1100', '1101', '1111', '1110', '1010', '1011', '1001', '1000']
# Gray code as integers
def gray_code_int(n):
    return np.array([i ^ (i >> 1) for i in range(2**n)], dtype=int)
code = gray_code_int(int(2 * np.log2(len(A))))      # [ 0  1  3  2  6  7  5  4 12 13 15 14 10 11  9  8]
control_wires = np.log2(code ^ np.roll(code, -1)).astype(int) # Identify which bit flips (CNOT is applied on these wires)
wire_map = {control_index : wire for control_index, wire in enumerate(wires_j + wires_i)} # Map bit indices to actual wire labels

def UA(thetas, control_wires, ancilla):                                       # Apply the sequence of controlled rotations and C-NOT gates
    for theta, control_index in zip(thetas, control_wires):
        qml.RY(2 * theta, wires=ancilla)
        qml.CNOT(wires=[wire_map[control_index]] + ancilla)
def UB(wires_i, wires_j):                                                     # Swap the two registers
    for w_i, w_j in zip(wires_i, wires_j):
        qml.SWAP(wires=[w_i, w_j])
def HN(input_wires):                                                          # Apply Hadamard to all qubits in input_wires 
    for w in input_wires:
        qml.Hadamard(wires=w)
     
dev2 = qml.device('default.qubit', wires=ancilla_wires + wires_i + wires_j)    # We construct the circuit using these oracles and draw it.
@qml.qnode(dev2)                                                               # Creates a function that runs on the device 'dev'
def circuit():
    HN(wires_i)
    qml.Barrier()                                                             # To separate the sections in the circuit
    UA(thetas, control_wires, ancilla_wires)
    qml.Barrier()
    UB(wires_i, wires_j)
    qml.Barrier()
    HN(wires_i)
    return qml.probs(wires=ancilla_wires + wires_i)

wire_order = ancilla_wires + wires_i[::-1] + wires_j[::-1]
U2 = qml.matrix(circuit, wire_order=wire_order)().real
print("Block-encoded A.2:\n",np.round(U2,4))
# The block-encoding scale is alpha = max singular value of A = ||A||max
alpha2 = np.linalg.norm(A, 2)
print("\nalpha (spect norm) =", alpha2)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = 2**s
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block2 = U2[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block2, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha2 * block2), 6)


Block-encoded A.2:
 [[ 0.75    0.      1.5    ... -0.25    0.      0.    ]
 [ 0.      0.25    0.     ... -0.      0.25    0.3536]
 [ 1.5     0.     -0.5    ...  0.25   -0.      0.25  ]
 ...
 [ 0.25    0.     -0.25   ...  0.25   -0.      0.    ]
 [-0.     -0.25   -0.     ...  0.     -0.5     0.    ]
 [-0.     -0.3536 -0.25   ... -0.     -0.      1.    ]]

alpha (spect norm) = 7.7812748581433455

Approx A/alpha block (real part):
[[ 0.75  0.    1.5   0.5 ]
 [ 0.    0.25  0.   -0.  ]
 [ 1.5   0.   -0.5  -0.  ]
 [ 0.5  -0.    0.    1.  ]]

alpha * block (should be ~ A):
[[ 6.  0. 12.  4.]
 [ 0.  2.  0. -0.]
 [12.  0. -4. -0.]
 [ 4. -0.  0.  8.]] 6


### 2.2 Sparse matrix (omitted because it does not apply here)

## 3. Defined function (Pennylane)

In [27]:
op = qml.BlockEncode(A, wires=range(3)) 
U3 = qml.matrix(op)
alpha3 = op.hyperparameters["norm"]

print("Block-encoded A.3:\n",np.round(U3,4))
print("\nalpha (spect norm) =", alpha3)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = A.shape[0]
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block3 = U3[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block3, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha3 * block3, 6))

Block-encoded A.3:
 [[ 4.230e-02+0.j      0.000e+00+0.j      8.450e-02+0.j
   2.820e-02+0.j      9.951e-01+0.j      0.000e+00+0.0002j
  -6.000e-04-0.j     -1.400e-03-0.j    ]
 [ 0.000e+00+0.j      1.410e-02+0.j      0.000e+00+0.j
   0.000e+00+0.0141j  0.000e+00-0.0002j  9.998e-01+0.j
   0.000e+00-0.j     -0.000e+00-0.0005j]
 [ 8.450e-02+0.j      0.000e+00+0.j     -2.820e-02+0.j
   0.000e+00+0.j     -6.000e-04+0.j      0.000e+00+0.j
   9.960e-01+0.j     -1.200e-03-0.j    ]
 [ 2.820e-02+0.j     -0.000e+00-0.0141j  0.000e+00+0.j
   5.630e-02+0.j     -1.400e-03+0.j     -0.000e+00+0.0005j
  -1.200e-03+0.j      9.979e-01+0.j    ]
 [ 9.951e-01+0.j      0.000e+00+0.0002j -6.000e-04-0.j
  -1.400e-03-0.j     -4.230e-02+0.j     -0.000e+00+0.j
  -8.450e-02+0.j     -2.820e-02+0.j    ]
 [ 0.000e+00-0.0002j  9.998e-01+0.j      0.000e+00-0.j
  -0.000e+00-0.0005j -0.000e+00+0.j     -1.410e-02+0.j
  -0.000e+00+0.j      0.000e+00-0.0141j]
 [-6.000e-04+0.j      0.000e+00+0.j      9.960e-01+0.j
  -1.200e-0

## 4. Definition of BE (Qiskit)

In [28]:
# Compute normalization factor alpha (largest singular value)
_, s_vals, _ = svd(A)
alpha4 = max(s_vals)
A_norm = A / alpha4

# Build U(A) in block form
I2 = np.eye(A.shape[0])
AA_dag = A_norm @ A_norm.conj().T
A_dagA = A_norm.conj().T @ A_norm

block_upper_right = sqrtm(I2 - AA_dag)
block_lower_left = sqrtm(I2 - A_dagA)

top = np.hstack([A_norm, block_upper_right])
bottom = np.hstack([block_lower_left, -A_norm.conj().T])
U4 = np.vstack([top, bottom])

print("Block-encoded A.4:\n",np.round(U4,4))
print("\nalpha (spect norm) =", alpha4)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = A.shape[0]
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block4 = U4[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block4, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha4 * block4, 6))


# # EXTRA: Make U exactly unitary (it is only approximately so due to numerical errors)
# def nearest_unitary(M):
#     U, _, Vh = svd(M)
#     return U @ Vh
# U5_approx = nearest_unitary(U5)
# print("\nBlock-encoded A.5 (with approximate nearest unitary):\n",np.real(np.round(U5_approx,4)))

Block-encoded A.4:
 [[ 0.3855+0.j      0.    +0.j      0.7711+0.j      0.257 +0.j
   0.2749-0.j     -0.    +0.0406j -0.1983-0.j     -0.2724-0.j    ]
 [ 0.    +0.j      0.1285+0.j      0.    +0.j      0.    +0.1285j
   0.    -0.0406j  0.9808-0.j      0.    -0.0131j  0.    -0.0563j]
 [ 0.7711+0.j      0.    +0.j     -0.257 +0.j      0.    +0.j
  -0.1983-0.j     -0.    +0.0131j  0.5081-0.j     -0.2042-0.j    ]
 [ 0.257 +0.j     -0.    -0.1285j  0.    +0.j      0.5141+0.j
  -0.2724-0.j     -0.    +0.0563j -0.2042-0.j      0.7308-0.j    ]
 [ 0.2749-0.j     -0.    +0.0406j -0.1983-0.j     -0.2724-0.j
  -0.3855+0.j     -0.    +0.j     -0.7711+0.j     -0.257 +0.j    ]
 [ 0.    -0.0406j  0.9808-0.j      0.    -0.0131j  0.    -0.0563j
  -0.    +0.j     -0.1285+0.j     -0.    +0.j      0.    -0.1285j]
 [-0.1983-0.j     -0.    +0.0131j  0.5081-0.j     -0.2042-0.j
  -0.7711+0.j     -0.    +0.j      0.257 +0.j     -0.    +0.j    ]
 [-0.2724-0.j     -0.    +0.0563j -0.2042-0.j      0.7308-0.j
  -0.25

In [ ]:
# CHECKS
data = {
    "Correctness (A vs (alpha * U_00)": [
        np.allclose(A, alpha1 * block1),
        np.allclose(A, alpha2 * block2),
        np.allclose(A, alpha3 * block3),
        np.allclose(A, alpha4 * block4),
    ],
    "U Shape": [
        U1.shape, U2.shape, U3.shape, U4.shape
    ],
    "U Hermitian": [
        np.allclose(U1, U1.conj().T),
        np.allclose(U2, U2.conj().T),
        np.allclose(U3, U3.conj().T),
        np.allclose(U4, U4.conj().T),
    ],
    "Unitary": [
        np.allclose(U1 @ U1.conj().T, np.eye(U1.shape[0])),
        np.allclose(U2 @ U2.conj().T, np.eye(U2.shape[0])),
        np.allclose(U3 @ U3.conj().T, np.eye(U3.shape[0])),
        np.allclose(U4 @ U4.conj().T, np.eye(U4.shape[0])),
    ]
}
df = pd.DataFrame(data, index=["U1", "U2", "U3", "U4"])
print(df)


    Correctness (A vs alpha * U_00)   U Shape  U Hermitian  Unitary
U1                             True  (64, 64)         True     True
U2                            False  (32, 32)        False    False
U3                             True    (8, 8)         True     True
U4                             True    (8, 8)         True     True
